# Deploy an image classification model to Azure

In [ ]:
%matplotlib notebook
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
 
import azureml
from azureml.core import Workspace, Run

# display the core SDK version number
print("TensorFlow Version: ", tf.__version__)
print("Azure ML SDK Version: ", azureml.core.VERSION)

## Initialize Workspace
Initialize a workspace object from persisted configuration.

In [ ]:
ws = Workspace.from_config()
print("Resource group: ", ws.resource_group)
print("Location: ", ws.location)
print("Workspace name: ", ws.name)

## Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

In [ ]:
import os

project_folder = '../projects/deploy_sklearn_mnist'
os.makedirs(project_folder, exist_ok=True)

## Register a model

In [ ]:
from azureml.core.model import Model

model_name = "sklearn_mnist"
model_path = "./resources/models/sklearn_mnist_model.pkl"

model = Model.register(
    model_path=model_path,
    model_name=model_name,
    tags={"data": "mnist", "model": "classification"},
    description="Mnist handwriting recognition",
    workspace=ws
)

## Download MNIST Dataset

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

# Supress warning and informational messages
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

mnist = input_data.read_data_sets("../MNIST_data/")

# Restore warning and informational messages
tf.logging.set_verbosity(old_v)

print('images shape:', mnist.train.images.shape)
print('labels shape:', mnist.train.labels.shape)

## Test model locally

### Predict test data

Feed the test dataset to the model to get predictions.

In [ ]:
import pickle
from sklearn.externals import joblib

clf = joblib.load(model_path)
y_hat = clf.predict(mnist.test.images)

###  Examine the confusion matrix

Generate a confusion matrix to see how many samples from the test set are classified correctly. Notice the mis-classified value for the incorrect predictions.

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mx = confusion_matrix(mnist.test.labels, y_hat)
print(conf_mx)
print('Overall accuracy:', np.average(y_hat == mnist.test.labels))

Use `matplotlib` to display the confusion matrix as a graph. In this graph, the X axis represents the actual values, and the Y axis represents the predicted values. The color in each grid represents the error rate. The lighter the color, the higher the error rate is. For example, many 5's are mis-classified as 3's. Hence you see a bright grid at (5,3).

In [ ]:
# normalize the diagnal cells so that they don't overpower the rest of the cells when visualized
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
np.fill_diagonal(norm_conf_mx, 0)

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)
cax = ax.matshow(norm_conf_mx, cmap=plt.cm.bone)
ticks = np.arange(0, 10, 1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(ticks)
ax.set_yticklabels(ticks)
fig.colorbar(cax)
plt.ylabel('true labels', fontsize=14)
plt.xlabel('predicted values', fontsize=14)
#plt.savefig(os.path.join(project_folder, 'conf.png'))
plt.show()

## Provision the AKS Cluster

In the meantime, let's spin up an Azure Kubernetes Services cluster...

This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

aks_name = 'myaks'

try:
    aks_target = AksCompute(workspace=ws, name=aks_name)
    print('found existing:', aks_target.name)
except ComputeTargetException:
    print('creating new.')

    # AKS configuration
    prov_config = AksCompute.provisioning_configuration(
        agent_count=2,
        vm_size="Standard_D3_v2"
    )
    
    # Create the cluster
    aks_target = ComputeTarget.create(
        workspace = ws, 
        name = aks_name, 
        provisioning_configuration = prov_config
    )

## Deploy as web service

Once you've tested the model and are satisfied with the results, deploy the model as a web service hosted in ACI. 

To build the correct environment for ACI, provide the following:
* A scoring script to show how to use the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model you trained before

### Create scoring script

Create the scoring script, called score.py, used by the web service call to show how to use the model.

You must include two required functions into the scoring script:
* The `init()` function, which typically loads the model into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.


In [ ]:
%%writefile score_mnist.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('sklearn_mnist')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

### Create environment file

Next, create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs `scikit-learn` and `azureml-sdk`.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

with open(os.path.join(project_folder, "myenv.yml"),"w") as f:
    f.write(myenv.serialize_to_string())

Review the content of the `myenv.yml` file.

In [ ]:
with open(os.path.join(project_folder, "myenv.yml"),"r") as f:
    print(f.read())

### Create ACI configuration

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. If you feel you need more later, you would have to recreate the image and redeploy the service.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1, 
    memory_gb=1, 
    tags={"data": "MNIST",  "method" : "sklearn"}, 
    description='Predict MNIST with sklearn'
)

### Deploy in ACI
Estimated time to complete: **about 7-8 minutes**

Configure the image and deploy. The following code goes through these steps:

1. Build an image using:
   * The scoring file (`score.py`)
   * The environment file (`myenv.yml`)
   * The model file
1. Register that image under the workspace. 
1. Send the image to the ACI container.
1. Start up a container in ACI using the image.
1. Get the web service HTTP endpoint.

In [ ]:
%%time
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(
    execution_script="score_mnist.py", 
    runtime="python", 
    conda_file=os.path.join(project_folder, "myenv.yml")
)

image = ContainerImage.create(
    name = "sklearn-mnist-img",
    models = [model],
    image_config = image_config,
    workspace = ws
)

image.wait_for_creation(show_output = True)

In [ ]:
%%time
from azureml.core.webservice import Webservice

aci_service = Webservice.deploy_from_image(
    workspace = ws, 
    name = 'sklearn-mnist-aci-svc',
    image = image,
    deployment_config = aciconfig
)

aci_service.wait_for_deployment(show_output=True)

Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [ ]:
print(aci_service.scoring_uri)

## Test deployed service

Earlier you scored all the test data with the local version of the model. Now, you can test the deployed model with a random sample of 30 images from the test data.  

The following code goes through these steps:
1. Send the data as a JSON array to the web service hosted in ACI. 

1. Use the SDK's `run` API to invoke the service. You can also make raw calls using any HTTP tool such as curl.

1. Print the returned predictions and plot them along with the input images. Red font and inverse image (white on black) is used to highlight the misclassified samples. 

 Since the model accuracy is high, you might have to run the following code a few times before you can see a misclassified sample.

In [ ]:
import json

# find 30 random samples from test set
n = 30
sample_indices = np.random.permutation(mnist.test.images.shape[0])[0:n]

test_samples = json.dumps({"data": mnist.test.images[sample_indices].tolist()})
test_samples = bytes(test_samples, encoding='utf8')

# predict using the deployed model
result = aci_service.run(input_data=test_samples)

# compare actual value vs. the predicted values:
i = 0
plt.figure(figsize = (20, 1))

for s in sample_indices:
    plt.subplot(1, n, i + 1)
    plt.axhline('')
    plt.axvline('')
    
    # use different color for misclassified sample
    font_color = 'red' if mnist.test.labels[s] != result[i] else 'black'
    clr_map = plt.cm.gray if mnist.test.labels[s] != result[i] else plt.cm.Greys
    
    plt.text(x=10, y =-10, s=result[i], fontsize=18, color=font_color)
    plt.imshow(mnist.test.images[s].reshape(28, 28), cmap=clr_map)
    
    i = i + 1
plt.show()

You can also send raw HTTP request to test the web service.

In [ ]:
import requests
import json

# send a random row from the test set to score
random_index = np.random.randint(0, len(mnist.test.images)-1)
input_data = "{\"data\": [" + str(list(mnist.test.images[random_index])) + "]}"

headers = {'Content-Type':'application/json'}

resp = requests.post(aci_service.scoring_uri, input_data, headers=headers)

print("POST to url", aci_service.scoring_uri)
#print("input data:", input_data)
print("label:", mnist.test.labels[random_index])
print("prediction:", resp.text)

## Deploy to Azure Kubernetes Services
### Check AKS Cluster state

In [ ]:
import time

status = aks_target.get_status()
while status != 'Succeeded' and status != 'Failed':
    print('current status: {} - waiting...'.format(status))
    time.sleep(10)
    status = aks_target.get_status()

In [ ]:
%%time
from azureml.core.webservice import Webservice, AksWebservice

aks_service_name = 'sklearn-mnist-aks-svc'

#Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration(
    cpu_cores=1, 
    memory_gb=1, 
    tags={"data": "MNIST",  "method" : "sklearn"}, 
    description='Predict MNIST with sklearn'
)

aks_service = Webservice.deploy_from_image(
    workspace = ws, 
    name = aks_service_name,
    image = image,
    deployment_config = aks_config,
    deployment_target = aks_target
)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

In [ ]:
import requests
import json

# send a random row from the test set to score
random_index = np.random.randint(0, len(mnist.test.images)-1)
input_data = "{\"data\": [" + str(list(mnist.test.images[random_index])) + "]}"

# for AKS deployment you'd need to the service key in the header as well
api_keys = aks_service.get_keys()
headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_keys[0])} 

resp = requests.post(aks_service.scoring_uri, input_data, headers=headers)

print("POST to url", aks_service.scoring_uri)
#print("input data:", input_data)
print("label:", mnist.test.labels[random_index])
print("prediction:", resp.text)

## Clean up resources

To keep the resource group and workspace for other tutorials and exploration, you can delete only the ACI deployment using this API call:

In [ ]:
aci_service.delete()
aks_service.delete()